In [0]:
import threading

def countdown():
     x = 1000000000
     while x > 0:
           x -= 1
# Implementation 1: Multi-threading
def implementation_1():
    thread_1 = threading.Thread(target=countdown)
    thread_2 = threading.Thread(target=countdown)
    thread_1.start()
    thread_2.start()
    thread_1.join()
    thread_2.join()
    return 'Termina Concurrente.'
# Implementation 2: Run in serial
def implementation_2():
    countdown()
    countdown()
    return 'Termina Serial.'
%timeit implementation_1()
%timeit implementation_2()

1 loop, best of 3: 1min 34s per loop
1 loop, best of 3: 1min 33s per loop


### Lock

In [0]:
import threading
import time
import random
import sys

#Thread-safe print
def sprint(s):
  sys.stdout.write(s + '\n')

In [0]:
#Clase contador
class Contador(object):
    def __init__(self, inicio = 0):
        self.lock = threading.Lock()
        self.value = inicio
    def incremento(self):
        sprint('Esperando por el lock')
        self.lock.acquire()
        try:
            sprint('Adquiero lock')
            self.value = self.value + 1
        finally:
            sprint('Libero lock')
            self.lock.release()
#Trabajador realizará tres veces: Dormir un poco y añadir uno al contador.
def trabajador(c):
    for i in range(3):
        r = random.random()
        sprint('Durmiendo ' + str(r))
        time.sleep(r)
        c.incremento()
    sprint('Listo')

if __name__ == '__main__':
    c = Contador()
    trabajadores = []
    for i in range(7):
        t = threading.Thread(target=trabajador, args=(c,))
        t.start()
        trabajadores.append(t)

    sprint('Esperando fin de trabajadores')
    for t in trabajadores:
      t.join()
    sprint('Contador: ' + str(c.value))

Durmiendo 0.8838449923062042
Durmiendo 0.9116178702030304
Durmiendo 0.6775452400507962
Durmiendo 0.053673961637487944
Durmiendo 0.6388850361720625
Durmiendo 0.7122106928960777
Durmiendo 0.22267129703136468
Esperando fin de trabajadores
Esperando por el lock
Adquiero lock
Libero lock
Durmiendo 0.8368767176859873
Esperando por el lock
Adquiero lock
Libero lock
Durmiendo 0.5638118046283883
Esperando por el lock
Adquiero lock
Libero lock
Durmiendo 0.7313007954806807
Esperando por el lock
Adquiero lock
Libero lock
Durmiendo 0.5603335984423097
Esperando por el lock
Adquiero lock
Libero lock
Durmiendo 0.7614876926405268
Esperando por el lock
Adquiero lock
Libero lock
Durmiendo 0.02527937365700683
Esperando por el lock
Adquiero lock
Libero lock
Listo
Esperando por el lock
Adquiero lock
Libero lock
Durmiendo 0.54406855515713
Esperando por el lock
Adquiero lock
Libero lock
Durmiendo 0.5977720364574451
Esperando por el lock
Adquiero lock
Libero lock
Durmiendo 0.7842432298071972
Esperando por el l

Una vez adquirido, el Lock previene la entrada a la sección crítica hasta que se libere.


Aunque es una solución buena, tiene sus potenciales problemas.
Veamos un ejemplo:

In [0]:
class Suma(object):
    def __init__(self,a,b):
      self.lock = threading.Lock()
      self.a = a
      self.b = b
      self.resultado = 0
    def establecerValor(self):
      self.lock.acquire()
      sprint('Esperando por el lock')
      try:
        sprint('Adquiero lock')
        v1 = self.getA()
        v2 = self.getB()
        self.resultado = v1+v2
      finally:
        sprint('Libero lock')
        self.lock.release()
    def getA(self):
      self.lock.acquire()
      sprint('Esperando por el lock')
      valor = None
      try:
        sprint('Adquiero lock')
        valor = a
      finally:
        sprint('Libero lock')
        self.lock.release()
      return valor
    def getB(self):
      self.lock.acquire()
      sprint('Esperando por el lock')
      valor = None
      try:
        sprint('Adquiero lock')
        valor = b
      finally:
        sprint('Libero lock')
        self.lock.release()
      return valor
  
s = Suma(12,13)
s.establecerValor()
print(s.resultado)

Como se puede ver, este código no puede finalizar. La solución se encuentra en utilizar un **RLock**.


In [0]:
class Suma2(object):
    def __init__(self,a,b):
      self.lock = threading.RLock()
      self.a = a
      self.b = b
      self.resultado = 0
    def establecerValor(self):
      self.lock.acquire()
      sprint('Esperando por el lock')
      try:
        sprint('Adquiero lock')
        v1 = self.getA()
        v2 = self.getB()
        self.resultado = v1+v2
      finally:
        sprint('Libero lock')
        self.lock.release()
    def getA(self):
      self.lock.acquire()
      sprint('Esperando por el lock')
      valor = None
      try:
        sprint('Adquiero lock')
        valor = self.a
      finally:
        sprint('Libero lock')
        self.lock.release()
      return valor
    def getB(self):
      self.lock.acquire()
      sprint('Esperando por el lock')
      valor = None
      try:
        sprint('Adquiero lock')
        valor = self.b
      finally:
        sprint('Libero lock')
        self.lock.release()
      return valor
  
s = Suma2(12,13)
s.establecerValor()
print(s.resultado)

El RLock previene solo a otros hilos a entrar al área crítica. Si es el mismo hilo, no existe problema.

**Semáforos**


Un semáforo se puede implementar con Semaphore y BoundedSempahore. Ambos son iguales pero si se intentan liberar recursos inexistentes, BoundedSemaphore te lo hará saber.

In [0]:
from threading import BoundedSemaphore
from threading import Thread

sprint("Bienvenidos a McMario's")

max_items = 10
container = BoundedSemaphore(max_items)

def producer(nloops):
    for i in range(nloops):
        time.sleep(random.randrange(1, 3))
        try:
            container.release()
            sprint("Se cocinó una MarioBurguesa.")
        except ValueError:
            sprint("[ERROR] El buffer está lleno. No se ha comido nada. :(")

def consumer(nloops):
    for i in range(nloops):
        time.sleep(random.randrange(2, 5))
        if container.acquire(False):
            sprint("Se comió una MarioBurguesa.")
        else:
            sprint("[ERROR] El buffer está vacío. La gente es adicta a las MarioBurguesas.")

threads = []
ploops = random.randrange(3, 6)
cloops = random.randrange(ploops, ploops+max_items+2)

sprint("Tamanio en almacén : " + str(max_items))
sprint("Numero de ciclos del cocinero: " + str(ploops))
sprint("Numero de visitas del comensal: " + str(cloops))

threads.append(Thread(target=producer, args=(ploops,)))
threads.append(Thread(target=consumer, args=(cloops,)))

for thread in threads:
    thread.start()
for thread in threads:
    thread.join()

sprint("Servidos y Contentos.")

Bienvenidos a McMario's
Tamanio en almacén : 10
Numero de ciclos del cocinero: 3
Numero de visitas del comensal: 8
[ERROR] El buffer está lleno. No se ha comido nada. :(
Se comió una MarioBurguesa.
Se cocinó una MarioBurguesa.
Se comió una MarioBurguesa.
Se cocinó una MarioBurguesa.
Se comió una MarioBurguesa.
Se comió una MarioBurguesa.
Se comió una MarioBurguesa.
Se comió una MarioBurguesa.
Se comió una MarioBurguesa.
Se comió una MarioBurguesa.
Servidos y Contentos.


Event

El objeto método, es la forma de comunicación más sencilla entre hilos. Event actua como una bandera que intercomunica hilos.

In [0]:
import threading
from threading import Event
import time

#Clase Reloj
class Reloj(threading.Thread):
  
    def __init__(self):
        threading.Thread.__init__(self)
        self.event = Event()
        self.segundo = 0

    def run(self):
        while self.event.is_set():
            print ("ID: ",threading.get_ident()," Numero: ",self.segundo)
            self.segundo += 1
            time.sleep(0.1)

In [0]:
hilo1 = Reloj()
hilo1.event.set()

hilo1.start()



ID:  139737380988672  Numero:  0


In [0]:
hilo1.event.clear()

ID:  139737380988672  Numero:  744
ID:  139737380988672  Numero:  745


In [0]:
print("Hola")

Hola
